### Generate_san_vw_with_rules
For train,test,dev files

In [1]:
# import fasttext
# model = fasttext.load_model("../../fastText/result/cc.sanskrit.50.bin")

In [2]:
import pandas as pd
# data = pd.read_csv('../../DCST/data/Rules/rule_'+'Gold_1496',header=None,sep=',')
# data

In [3]:
# load = pd.read_csv('../../DCST/data/sanskrit_treebank/'+'dev/'+'Gold_1496'+'.csv',header=None,sep=',')
# load

In [4]:
def get_rule(load,file, line_no):
#     print('function: ',file,line_no)
    head=''
    pos=''
    tag=''
    try:
        df = pd.read_csv('../../DCST/data/Rules/rule_'+file,header=None,sep=',')
    except:
        return head,pos,tag
    for i in range(len(df)):
        if df.iloc[i,1] == int(line_no):
            head+= ' '+str(load.iloc[df.iloc[i,4],1])
            pos+= ' '+str(load.iloc[df.iloc[i,4],3])
            tag+= ' '+str(df.iloc[i,8])
    return head,pos,tag

    
# print(get_rule(load,'Gold_1496',2))

In [5]:

# Get all the tags in hash list
hash = {}
tag_file_name = '../data/san_tags'
# input_file_name = './sanskrit_treebank/ud_pos_ner_dp_train_san'
files = ['ud_pos_ner_dp_dev_san','ud_pos_ner_dp_train_san','ud_pos_ner_dp_test_san']

def readtags():
    for line in open(tag_file_name).readlines():
        hash[line.split()[0]] = int(line.strip().split()[1])
c = 1
readtags()
for input_file_name in files:
    if 'train' in input_file_name:
        target = 'train/'
    elif 'test' in input_file_name:
        target = 'test/'
    else:
        target = 'dev/'
    data = open('../data/'+input_file_name).readlines()
    writer = open('../models/Dep_san_temp/'+input_file_name+'_with_rules'+'.vw','w')
    for line in data:
#         print('Line',line.replace('\n',''))
        if line == '\n':
            writer.write('\n')
            continue
        splits = line.strip().split('\t')

        # This replace can be removed.
        strw = "|w %s"%splits[1].replace(":","COL");
        strp = "|p %s"%splits[2].replace(":","COL");
#         
#         print(head,tag)
        ##############################################################
        # To pass gold data use this blog
        if 'test' in input_file_name:
            load = pd.read_csv('../../DCST/data/sanskrit_treebank/'+'test/'+splits[3]+'.csv',header=None,sep=',')
        else:
            try:
                load = pd.read_csv('../../DCST/data/sanskrit_treebank/'+'train/'+splits[3]+'.csv',header=None,sep=',')
            except:
                load = pd.read_csv('../../DCST/data/sanskrit_treebank/'+'dev/'+splits[3]+'.csv',header=None,sep=',')
#         print(load.iloc[int(splits[4])-1,1],load.iloc[int(splits[4])-1,3])
#         if int(splits[4]) != 0:
#             hrule= "|hrule %s"%str(load.iloc[int(splits[4]),1])
#             prule= "|prule %s"%str(load.iloc[int(splits[4]),3])
#         else:
#             hrule= "|hrule %s"%splits[1]
#             prule= "|prule %s"%splits[2]
#         ttule= "|trule %s"%splits[-1]
        ###############################################################
        # To pass samsadhini output use this function
        head,pos,tag =get_rule(load,splits[3],splits[0])
        h = "|hrule %s"%head;
        p = "|prule %s"%pos;
        t = "|trule %s"%tag;
#         print(hrule)
#         word = model.get_word_vector(splits[1]);
        feature = ''
        i=0
#         for v in word:
#             feature = feature+' '+str(v)
# #             feature = feature+' '+'F'+str(i)+':'+str(v)

#             i=i+1
#         embedding = "|FastText%s"%feature;
#         strq = "|FASTTEXT T0:%s U1:%s"%(word[1],word[1]);
        tag = splits[-1]
        if tag not in hash:
            hash[tag] = c
            c+=1

        #writer.write('%s 1.0  %s:%s%s %s\n'%((int(splits[7])+1) + (hash[tag]<<8), int(splits[7]),tag,strw, strp))
        # Writing in order
        # head, tag_integer mapping, head, tag, word, pos
        writer.write('%s %s %s:%s%s %s %s %s %s\n' % (int(splits[4]), hash[tag], int(splits[4]), tag, strw, strp, h, p, t))
    writer.close()
print('Value of out of voc tag ',c)
print('successful')

Value of out of voc tag  1
successful
